In [1]:
import numpy as np
import pygame as pg
import matplotlib.pyplot as plt
import random

xs = np.linspace(-1, 1, 9)
ys = np.linspace(-1, 1, 9)
zs = np.linspace(-1, 1, 9)


rot_x = 0
rot_y = 0
rot_z = 0

rotv_x = 0
rotv_y = 0
rotv_z = 0


vectors = np.array([[0, 0, 0]])
for x in xs:
    for y in ys:
        for z in zs:
            vectors = np.append(vectors, np.array([[x, y, z]]), axis=0)
            #vectors = np.append(vectors, np.array([[x, np.sin(x**2 + z**2)*0.2, z]]), axis=0)


running = True
pg.init()
width, height = 800, 800
bg_color = (0, 0, 0)
screen = pg.display.set_mode((width, height))
clock = pg.time.Clock()
rot = 0
show_transformation = False

def rotate_z(vec, rotation):
    rotmat = np.array([[np.cos(rotation), -np.sin(rotation), 0],
    [np.sin(rotation), np.cos(rotation), 0],
    [0, 0, 1]])
    return np.matmul(rotmat, vec)

def rotate_y(vec, rotation):
    rotmat = np.array(
            [[np.cos(rotation), 0, np.sin(rotation)],
            [0, 1, 0],
            [-np.sin(rotation), 0, np.cos(rotation)]])
    return np.matmul(rotmat, vec)

def rotate_x(vec, rotation):
    rotmat = np.array([
            [1, 0, 0],
            [0, np.cos(rotation), -np.sin(rotation)],
            [0, np.sin(rotation), np.cos(rotation)]])
    return np.matmul(rotmat, vec)

projection_matrix = np.array([[1,0,0],[0,1,0]])
scale = 2**(1/3)*0.5
scale_matrix = np.array([[scale, 0, 0], [0, scale, 0], [0, 0, scale]])
mouse_down = False

mouse_x = 0
mouse_y = 0

while running:
    dx, dy = pg.mouse.get_rel()
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
        if event.type == pg.MOUSEBUTTONDOWN:
            mouse_down = True
        if event.type == pg.MOUSEBUTTONUP:
            mouse_down = False
        if event.type == pg.MOUSEMOTION:
            mouse_x, mouse_y = event.pos
        if event.type == pg.KEYDOWN:
            if event.key == pg.K_ESCAPE:
                running = False
            if event.key == pg.K_m:
                show_transformation = not show_transformation

    if mouse_down:
        rot_y += dx*0.005
        rot_x += dy*0.005
        rotv_y += dx*0.001
        rotv_x += dy*0.001    
    screen.fill(bg_color)
    ball_index = 0
    rot_x += rotv_x
    rot_y += rotv_y
    rot_z += rotv_z
    rotv_x *= 0.9
    rotv_y *= 0.9
    rotv_z *= 0.9


    for point in vectors:
        line_size = 0
        ball_color = (point+1)*120
        scaled_point = np.matmul(scale_matrix, point)
        rpoint = rotate_y(scaled_point, rot_y)
        rpoint = rotate_x(rpoint, rot_x)
        rpoint = rotate_z(rpoint, rot_z)
        
        z = 2/(2+rpoint[2])
        ppoint = np.matmul(projection_matrix, rpoint)
        x, y = (ppoint[0])*width/2 + width/2, (ppoint[1])*height/-2 + height/2
        x*=z
        y*=z
        ##here we find distancio
        distanceroni = ((mouse_x-x)**2 + (mouse_y-y)**2)**0.5
        distanceroni /= np.linalg.norm(np.array([width, height]))
        pg.draw.circle(screen, ball_color*((1-distanceroni)**4), (x, y), 0 + int(9*((1-distanceroni)**4)), line_size)
        if show_transformation:
            po = np.matmul(projection_matrix, scaled_point)
            x0, y0 = (po[0])*width/2 + width/2, (po[1])*height/-2 + height/2
            pg.draw.line(screen, ball_color, (x0, y0), (x, y), 1)
        ball_index+= 1
    pg.display.flip()
    clock.tick(30)
pg.quit()

pygame 2.0.1 (SDL 2.0.14, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
